In [1]:
!pip install librosa
!pip install transformers==4.28.0 datasets evaluate
!pip install torch
!pip install torchaudio

In [3]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
# from custom_dataset import (CustomDataSet,
#                             speech_file_to_array_fn as sfaf,
#                             collate_fn,
#                             create_data_loader,
#                             validation,
#                             train)
# from transformers import (Wav2Vec2FeatureExtractor,
#                           Wav2Vec2Model,
#                           Wav2Vec2Config,
#                           Wav2Vec2ConformerForSequenceClassification,
#                           AutoModelForAudioClassification)
import evaluate
import librosa
import random
# import torch
# import torchaudio
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.nn.utils.rnn import pad_sequence 
# from torch.utils.data import Dataset, DataLoader
from sklearn.tree import DecisionTreeClassifier
# from huggingface_hub import notebook_login
from datasets import load_dataset, Audio
# import warnings
# warnings.filterwarnings(action='ignore')
# os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [4]:
CFG = {
    'SR':16000,
    'N_MFCC':32, # Melspectrogram 벡터를 추출할 개수
    'SEED':42
}

In [5]:
CFG = {
    'SR':16_000,
    'BATCH_SIZE' : 8,
    'TOTAL_BATCH_SIZE': 42,
    'N_MFCC':32, # Melspectrogram 벡터를 추출할 개수
    'SEED':42,
    'EPOCHS' : 1,
    'LR' : 1e-4
}

{'SR': 16000, 'N_MFCC': 32, 'SEED': 42}

In [6]:
train_df = pd.read_csv('./train.csv')
print(train_df.info())
print()
test_df = pd.read_csv('./test.csv')
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001 entries, 0 to 5000
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5001 non-null   object
 1   path    5001 non-null   object
 2   label   5001 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 117.3+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1881 entries, 0 to 1880
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1881 non-null   object
 1   path    1881 non-null   object
dtypes: object(2)
memory usage: 29.5+ KB
None


In [7]:
# Folder Locations

dataset = "./"
TRAIN_WAV = dataset + "train/"
TEST_WAV = dataset + "test/"
PREPROCESSED = dataset + "preprocessed_data/"
TRAIN_LABEL_SEP = PREPROCESSED + "train_label_sep/"
WAV_TRAIN_LABEL_SEP = PREPROCESSED + "wav_train_label_sep/"
TEST_LABEL_SEP = PREPROCESSED + "test_label_sep/"
WAV_TEST_LABEL_SEP = PREPROCESSED + "wav_test_label_sep/"


if not os.path.exists(dataset + "preprocessed_data"):
    os.mkdir(dataset + "preprocessed_data")
    
if not os.path.exists(PREPROCESSED + "train_label_sep"):
    os.mkdir(PREPROCESSED + "train_label_sep")

if not os.path.exists(PREPROCESSED + "test_label_sep"):
    os.mkdir(PREPROCESSED + "test_label_sep")

if not os.path.exists(PREPROCESSED + "wav_train_label_sep"):
    os.mkdir(PREPROCESSED + "wav_train_label_sep")

if not os.path.exists(PREPROCESSED + "wav_test_label_sep"):
    os.mkdir(PREPROCESSED + "wav_test_label_sep")

In [8]:
wav_file_dict = {"train_wav" : TRAIN_WAV,
                "test_wav" : TEST_WAV,
                 "wav_sep" : WAV_TRAIN_LABEL_SEP
                 }

wav_file_locations = {}
for key, value in wav_file_dict.items():
    wav_file_locations[key] = glob.glob(value + "*.wav")
    
csv_file_dict = {"train_label_sep" : TRAIN_LABEL_SEP,
                 "wav_train_label_sep" : WAV_TRAIN_LABEL_SEP
                }

csv_file_location = {}
for key, value in csv_file_dict.items():
    csv_file_location[key] = glob.glob(value + "*.csv")

In [9]:
def get_mfcc_feature(df):
    features = []
    for path in tqdm(df['path']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=CFG['SR'])
        # librosa패키지를 사용하여 mfcc 추출
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])
        y_feature = []
        # 추출된 MFCC들의 평균을 Feature로 사용
        for e in mfcc:
            y_feature.append(np.mean(e))
        features.append(y_feature)

    mfcc_df = pd.DataFrame(features, columns=['mfcc_'+str(x) for x in range(1,CFG['N_MFCC']+1)])
    return mfcc_df


In [10]:
train_x = get_mfcc_feature(train_df)
print(train_x)
test_x = get_mfcc_feature(test_df)
print(test_x)

  0%|          | 0/5001 [00:00<?, ?it/s]

          mfcc_1      mfcc_2     mfcc_3     mfcc_4     mfcc_5    mfcc_6  \
0    -414.755737  110.100639  46.699074  23.939814  14.766221  4.820827   
1    -399.769531   83.051300  55.473316  31.782587  22.040754  0.985082   
2    -341.145081   97.399071  38.274349  19.811539   0.731027  0.838704   
3    -376.963715  118.961670  34.490349  24.178417  -1.065604 -1.613391   
4    -352.863220  117.553337  29.948687  31.094315   5.406391 -5.591998   
...          ...         ...        ...        ...        ...       ...   
4996 -416.181305  112.938484  47.294231  23.111433  16.048231  6.795750   
4997 -237.811432   72.207787   3.371584  14.646128 -10.546066  7.671333   
4998 -368.228119  103.887871  40.081417  21.763754  14.120455  5.778781   
4999 -407.114288  103.880676  44.980690  23.265039  13.279220  6.904113   
5000 -359.754608   89.241714  20.517361  24.043827  16.926319 -7.636394   

        mfcc_7     mfcc_8    mfcc_9    mfcc_10  ...   mfcc_23   mfcc_24  \
0     1.445079  -0.92615

  0%|          | 0/1881 [00:00<?, ?it/s]

          mfcc_1      mfcc_2     mfcc_3     mfcc_4     mfcc_5    mfcc_6  \
0    -335.757324  125.215431  22.145767  14.351713  -1.045251  0.567860   
1    -295.973053   92.839684  24.976181  22.831310 -10.278670  7.804742   
2    -444.395996  109.385201  55.236771  26.486050  12.487712  9.069915   
3    -384.600220  112.463974  47.454556  17.336460  13.556947  4.677102   
4    -273.304077   97.817047  12.370095  24.591480   3.248469 -9.987856   
...          ...         ...        ...        ...        ...       ...   
1876 -250.254913   69.372955  17.328987  12.005389  -2.463175 -7.652928   
1877 -348.593842  107.067047  27.683287  17.207047   2.634121  1.812742   
1878 -295.658112   99.606911  -9.669126   4.676853   4.752311 -5.689676   
1879 -450.677094  122.865677  46.413559  21.616255  13.785479  5.433998   
1880 -366.914154  113.387276  43.263287  15.806440   8.695131  6.917760   

        mfcc_7     mfcc_8    mfcc_9    mfcc_10  ...   mfcc_23   mfcc_24  \
0    -0.666742  -8.11483

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic=True
    torch.backends.cudnn.benchmark=True

seed_everything(CFG['SEED'])

In [11]:
print(train_x.info())
print()
print()
print()
print(test_x.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001 entries, 0 to 5000
Data columns (total 32 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   mfcc_1   5001 non-null   float32
 1   mfcc_2   5001 non-null   float32
 2   mfcc_3   5001 non-null   float32
 3   mfcc_4   5001 non-null   float32
 4   mfcc_5   5001 non-null   float32
 5   mfcc_6   5001 non-null   float32
 6   mfcc_7   5001 non-null   float32
 7   mfcc_8   5001 non-null   float32
 8   mfcc_9   5001 non-null   float32
 9   mfcc_10  5001 non-null   float32
 10  mfcc_11  5001 non-null   float32
 11  mfcc_12  5001 non-null   float32
 12  mfcc_13  5001 non-null   float32
 13  mfcc_14  5001 non-null   float32
 14  mfcc_15  5001 non-null   float32
 15  mfcc_16  5001 non-null   float32
 16  mfcc_17  5001 non-null   float32
 17  mfcc_18  5001 non-null   float32
 18  mfcc_19  5001 non-null   float32
 19  mfcc_20  5001 non-null   float32
 20  mfcc_21  5001 non-null   float32
 21  mfcc_22  5001 

In [ ]:
train_y = train_df['label']
print(train_y)

In [ ]:
model = DecisionTreeClassifier(random_state=CFG['SEED'])
model.fit(train_x, train_y)

In [ ]:
preds = model.predict(test_x)

In [ ]:
submission = pd.read_csv(dataset + 'sample_submission.csv')
submission['label'] = preds
submission.to_csv(dataset + "baseline_submission.csv", index=False)

In [ ]:
test_dataset = CustomDataSet(test_x, y=None, processor=processor)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, collate_fn=collate_fn_test)

In [ ]:
def inference(model, test_loader):
    model.eval()
    preds = []

    with torch.no_grad():
        for x in tqdm(iter(test_loader)):
            x = x.to(device)

            output = model(x)

            preds += output.argmax(-1).detach().cpu().numpy().tolist()

    return preds

In [ ]:
preds = inference(infer_model, test_loader)

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission['label'] = preds
submission.to_csv('./baseline_submission.csv', index=False)